In [1]:
from typing import List, Optional
from langchain_core.embeddings import Embeddings
import requests

class OllamaEmbeddings(Embeddings):
    def __init__(self, model_name: str = "nomic-embed-text", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = []
        for text in texts:
            response = requests.post(
                f"{self.base_url}/api/embeddings",
                json={"model": self.model_name, "prompt": text}
            )
            if response.status_code == 200:
                embeddings.append(response.json()["embedding"])
            else:
                raise ValueError(f"Embedding error: {response.text}")
        return embeddings

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

In [ ]:
from langchain_core.language_models import BaseLLM
from langchain_core.messages import HumanMessage
from typing import Any, Dict, List, Optional
import requests

class CustomHTTPLLM(BaseLLM):
    api_url: str = "http://localhost:11434/api/chat"
    model_name: str = "qwen2:latest"
    temperature: float = 0.7
    max_tokens: int = 100

    def _generate(self, messages: List[HumanMessage], **kwargs:Any) -> str:  # 已修复此行
        formatted_messages = [{"role": "user", "content": msg.content} for msg in messages]
        
        response = requests.post(
            self.api_url,
            json={
                "model": self.model_name,
                "messages": formatted_messages,
                "stream": False,
                "options": {
                    "temperature": self.temperature,
                    "max_tokens": self.max_tokens
                }
            }
        )
        
        if response.status_code == 200:
            return response.json()["message"]["content"]
        else:
            raise ValueError(f"API调用失败: {response.text}")

    @property
    def _llm_type(self) -> str:
        return "custom_http_llm"

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


# 加载文档
loader = TextLoader("./yanbao000.txt")
documents = loader.load()

# 分割文本
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(documents)

# 初始化嵌入模型
embeddings = OllamaEmbeddings(model_name="nomic-embed-text")

# 构建向量数据库
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 配置提示模板
template = """基于以下上下文回答问题：
{context}

问题：{question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 初始化 LLM
llm = CustomHTTPLLM()

# 构建 RAG 链
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# 提问
response = rag_chain.invoke("LangChain 的主要用途是什么？")
print(response)

AttributeError: 'str' object has no attribute 'content'